# 1. Libraries

In [1]:
# Import Libraries
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# 2. Introduction

The purpose of this notebook is to gather data on the European ARMS Challenge (EUAC) tournament series. <br>
The EUAC is an online biweekly tournament series for the video game "ARMS" released for the Nintendo Switch console in 2017. <br>
The EUAC started in 2017 and ran consistently until 2022.<br>
<br>
More information can be found in references below.

This notebook is to purely gather the data, act as a proof of concept, and format it for future use/analysis. <br>
It will then be exported to a csv file if successful. <br>

# 3. Objectives

- Determine what data that is necessary
- Gather EUAC#1 data from start.gg using their api
- Gather EUAC#2 to EUAC#110 data from Challonge.com using their api
- Format the data into a pandas dataframe
- Export to csv

## 3.1. Objective 1

### Determine what data is necessary

For each tournament, we will need: <br>
- The participants who entered
- Each match that was played
- The result of each match
- The date the tournament took place
- The final placements
- Each player's seeding

## 3.2. Objective 2 - Data Gathering

The tournaments were mostly hosted on Challonge.com but the first one was hosted on start.gg. <br>

### 3.2.1. - Start.gg

Start.gg's API uses GraphQL as its query language <br>
The GraphQL queries will be wrapped in multiline strings so they can be sent to the start.gg API to retrieve the data. <br>
Queries also expect an API token to passed through the header. This token will be loaded from a .env file.<br>
<br>
A link to the tutorial that was followed to come up with the queries can be found in References.

In [2]:
# Load API Keys stored in .env file
load_dotenv(".env")
START_GG_API_TOKEN = os.getenv("START_GG_API_TOKEN")

The tournament was hosted at this url: https://www.start.gg/tournament/eu-arms-challenge-1/events <br>
For some of the information that is needed, the id of the event is required. Which is obtained here using the url slug

In [31]:
# GraphQL query wrapped in a multiline string. 
query = """
query TournamentQuery {
  tournament(slug: "tournament/eu-arms-challenge-1") {
    name
    events {
      name
      id
    }
  }
}
"""

In [32]:
url = "https://api.start.gg/gql/alpha"
headers = {
    "Authorization": f"Bearer {START_GG_API_TOKEN}",
    "Content-Type": "application/json"
}
response = requests.post(url, json={'query': query}, headers=headers)
data = response.json()

In [33]:
data

{'data': {'tournament': {'name': 'EU ARMS Challenge #1',
   'events': [{'name': 'EU ARMS Challenge #1', 'id': 53835}]}},
 'extensions': {'cacheControl': {'version': 1,
   'hints': [{'path': ['tournament'], 'maxAge': 300, 'scope': 'PRIVATE'}]},
  'queryComplexity': 2},
 'actionRecords': []}

In [34]:
eventId = data["data"]["tournament"]["events"][0]["id"]

In [35]:
eventId

53835

The id of the event is 53835

Using the event id, a query can be written to get the players, their seedings and placements, and the start date of the tournament <br>
Some of this information could have been gotten before but not all

In [36]:
# Query for players, seedings, placements

query = """
query {
  event(id: 53835) {
    id
    name
    startAt
    standings(query: {
      page: 1
    }) {
      nodes {
        placement
        entrant {
          name
          seeds {
            seedNum
          }
        }
      }
    }
  }
}
"""

In [37]:
url = "https://api.start.gg/gql/alpha"
headers = {
    "Authorization": f"Bearer {START_GG_API_TOKEN}",
    "Content-Type": "application/json"
}
response = requests.post(url, json={'query': query}, headers=headers)
data = response.json()

In [38]:
data

{'data': {'event': {'id': 53835,
   'name': 'EU ARMS Challenge #1',
   'startAt': 1508605200,
   'standings': {'nodes': [{'placement': 1,
      'entrant': {'name': 'FR | Maxou0708', 'seeds': [{'seedNum': 20}]}},
     {'placement': 2,
      'entrant': {'name': 'Rapha_MTH', 'seeds': [{'seedNum': 6}]}},
     {'placement': 3,
      'entrant': {'name': 'Sabaca', 'seeds': [{'seedNum': 8}]}},
     {'placement': 4,
      'entrant': {'name': 'TCM | Raffa', 'seeds': [{'seedNum': 21}]}},
     {'placement': 5,
      'entrant': {'name': 'FrankTank', 'seeds': [{'seedNum': 19}]}},
     {'placement': 5, 'entrant': {'name': 'ocrim', 'seeds': [{'seedNum': 2}]}},
     {'placement': 7,
      'entrant': {'name': 'Alumento', 'seeds': [{'seedNum': 11}]}},
     {'placement': 7,
      'entrant': {'name': 'SC☆Momso', 'seeds': [{'seedNum': 7}]}},
     {'placement': 9,
      'entrant': {'name': 'VilleViljar', 'seeds': [{'seedNum': 4}]}},
     {'placement': 9,
      'entrant': {'name': 'TCM | BD82', 'seeds': [{'se

Data is a nested dictionary. Now we'll gather a list of participants, their placements and seeds, by digging through this

In [40]:
data["data"]["event"]["name"]

'EU ARMS Challenge #1'

In [43]:
data["data"]["event"]["startAt"]

1508605200

In [71]:
tournamentDate = data["data"]["event"]["startAt"]

UNIX time for when tournament took place

In [44]:
data["data"]["event"]["standings"]

{'nodes': [{'placement': 1,
   'entrant': {'name': 'FR | Maxou0708', 'seeds': [{'seedNum': 20}]}},
  {'placement': 2,
   'entrant': {'name': 'Rapha_MTH', 'seeds': [{'seedNum': 6}]}},
  {'placement': 3, 'entrant': {'name': 'Sabaca', 'seeds': [{'seedNum': 8}]}},
  {'placement': 4,
   'entrant': {'name': 'TCM | Raffa', 'seeds': [{'seedNum': 21}]}},
  {'placement': 5,
   'entrant': {'name': 'FrankTank', 'seeds': [{'seedNum': 19}]}},
  {'placement': 5, 'entrant': {'name': 'ocrim', 'seeds': [{'seedNum': 2}]}},
  {'placement': 7,
   'entrant': {'name': 'Alumento', 'seeds': [{'seedNum': 11}]}},
  {'placement': 7, 'entrant': {'name': 'SC☆Momso', 'seeds': [{'seedNum': 7}]}},
  {'placement': 9,
   'entrant': {'name': 'VilleViljar', 'seeds': [{'seedNum': 4}]}},
  {'placement': 9,
   'entrant': {'name': 'TCM | BD82', 'seeds': [{'seedNum': 17}]}},
  {'placement': 9,
   'entrant': {'name': 'FR|TCM | InkAlyut', 'seeds': [{'seedNum': 13}]}},
  {'placement': 9, 'entrant': {'name': 'Linkirto', 'seeds': [

In [49]:
data["data"]["event"]["standings"]["nodes"][0] #Information for one player

{'placement': 1,
 'entrant': {'name': 'FR | Maxou0708', 'seeds': [{'seedNum': 20}]}}

In [57]:
# Placement
data["data"]["event"]["standings"]["nodes"][0]["placement"]

1

In [60]:
# Name
data["data"]["event"]["standings"]["nodes"][0]["entrant"]["name"]

'FR | Maxou0708'

In [63]:
# Seeding
data["data"]["event"]["standings"]["nodes"][0]["entrant"]["seeds"][0]["seedNum"]

20

In [65]:
# Putting it all together in a for loop. Store information in arrays

playerArray = []
placementArray = []
seedArray = []

for entrant in data["data"]["event"]["standings"]["nodes"]:
    playerArray.append(entrant["entrant"]["name"])
    placementArray.append(entrant["placement"])
    seedArray.append(entrant["entrant"]["seeds"][0]["seedNum"])

In [69]:
# Make a pandas dataframe of the arrays

playerdf = pd.DataFrame({
    "Player": playerArray,
    "Seed": seedArray,
    "Placement": placementArray
})

In [70]:
playerdf.head()

,Player,Seed,Placement
0,FR | Maxou0708,20,1
1,Rapha_MTH,6,2
2,Sabaca,8,3
3,TCM | Raffa,21,4
4,FrankTank,19,5


All players, their placements, and seeding, gathered. <br>To note for later: players can sign up with "tags". Denoted with a |. But | can be present in a tag

### 3.2.2 - Challonge.com

# References

- https://armswiki.org/wiki/EU_ARMS_Challenge
- https://developer.start.gg/docs/examples/queries/get-event

In [1]:
x="""query getEventId($slug: String) {
  tournament(slug: $slug) {
    name
    id
    events {
      name
      id
      startAt
      entrants {
        nodes {
          name
          seeds{
            seedNum
          }
        }
      }
    }
  }
}"""

In [2]:
y="""query EventStandings($eventId: ID!, $page: Int!) {
  event(id: $eventId) {
    id
    name
    startAt
    standings(query: {
      page: $page
    }){
      nodes {
        placement
        entrant {
          name
          seeds{
            seedNum
          }
        }
      }
    }
  }
}"""

In [3]:
z="""{
  "slug": "tournament/eu-arms-challenge-1",
  "eventId": 53835,
  "page": 1,
  "perPage": 30
}"""

In [4]:
a="""query set($setId: ID!) {
  set(id: $setId) {
    id
    slots {
      id
      standing {
        id
        placement
        stats {
          score {
            label
            value
          }
        }
      }
    }
  }
}"""